In [ ]:
%matplotlib inline

In [ ]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import json
import operator 
from collections import Counter
from nltk.corpus import stopwords
import string
import nltk
import pandas as pd
from nltk import bigrams 
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:

 
with open('bitcoin.json', 'r') as f:
    line = f.readline() # read only the first tweet/line
    #tweet = json.loads(line) # load it as Python dict
    #print(json.dumps(tweet, indent=4)) # pretty-print
    print(line)

In [ ]:

 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
    

In [ ]:
listtweets = []
with open('bitcoin.json', 'r') as f:
    for line in f:
        i = 0
        tweet = line
        tokens = preprocess(tweet)
        listtweets.append(tokens)
        
len(listtweets)

In [ ]:
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt','RT', 'via', '...']

In [ ]:
fname = 'bitcoin.json'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet =line
        terms_stop = [term for term in preprocess(tweet) if term not in stop]
        terms_bigram = bigrams(terms_stop)
        # Update the counter
        count_all.update(terms_bigram)
    # Print the first 5 most frequent words
    print(count_all.most_common(10))

In [ ]:
terms_stop

In [ ]:
x = pd.DataFrame(count_all.most_common(20))
x

In [ ]:
# n_docs is the total n. of tweets
n_docs = len(listtweets)
p_t = {}
p_t_com = defaultdict(lambda : defaultdict(int))
 
for term, n in count_all.items():
    p_t[term] = n / n_docs
    for t2 in listtweets:
        p_t_com[term][t2] = listtweets[term][t2] / n_docs

In [ ]:
positive_vocab = [
    'good', 'nice', 'great', 'awesome', 'outstanding',
    'fantastic', 'terrific', ':)', ':-)', 'like', 'love',
    # shall we also include game-specific terms?
    # 'triumph', 'triumphal', 'triumphant', 'victory', etc.
]
negative_vocab = [
    'bad', 'terrible', 'crap', 'useless', 'hate', ':(', ':-(',
    # 'defeat', etc.
]

In [ ]:
from os import path
from wordcloud import WordCloud

#d = path.dirname(__file__)

# Read the whole text.
text = open( 'bitcoin.json').read()

# Generate a word cloud image
wordcloud = WordCloud().generate(text)

# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# lower max_font_size
wordcloud = WordCloud(max_font_size=40).generate(text)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.show()

In [ ]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
 
consumer_key = 'nYxXmA9B7MmpgRdQBzg9LZxCp'
consumer_secret = 'cE5yAit3vEAIyQIyM69uGuSMAxjh9y8cPWXu7lyR8HF0EB02D4'
access_token = '45810954-CGIPoVv7cSkYLG5XsXFXLv2B4p5yfLfg3uF7EFUcc'
access_secret = '4SzUyUbAbTePDOjet7cw6AAOWwV41Wy9mZzdJV6TCpgYT'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)
listy = []

 
class MyListener(StreamListener):
    def on_status(self, status):
        listy.append(status)
        print(status.text)
        print("foo")
        print_ratios(listy)
        print(status.created_at,status.text)
    
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False
        

class MySaver(StreamListener): 
    def on_status(self, data):
        try:
            with open('bitcoin.json', 'a') as f:
                f.write(data.text)
                return True
        except BaseException as e:
            print(str(e))
        return True
 
    def on_error(self, status):
        #print(status)
        return True
 
twitter_stream = Stream(auth, MyListener())
#twitter2_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#bitcoin'], async=True)
#twitter2_stream.filter(track=['#ether'], async=True)

In [ ]:
len(listy)

In [ ]:
def get_tweet_sentiment(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_tweet(tweet))
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative',analysis.sentiment.polarity

In [ ]:
def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [ ]:
def get_tweet(listy):
    tweets = []
    for tweet in listy:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
    
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = get_tweet_sentiment(tweet.text)
    
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
    return(tweets)

In [1]:
tweets = get_tweet(listy)

NameError: name 'get_tweet' is not defined

In [ ]:
tweets

In [ ]:
def print_ratios(listy):
    tweets = get_tweet(listy)
    print(len(tweets))
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    print("Positive: {} %".format(100*len(ptweets)/len(tweets)),
      "Negative: {} %".format(100*len(ntweets)/len(tweets)),
      "Neutral:{}%".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets))
     )


In [ ]:
print("Positive: {} %".format(100*len(ptweets)/len(tweets)),
      "Negative: {} %".format(100*len(ntweets)/len(tweets)),
      "Neutral:{}%".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets))
     )
print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
print("Neutral tweets percentage:{}%".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))